<a href="https://colab.research.google.com/github/chetools/chetools/blob/main/tools/DelayedTFSimulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00


In [ ]:
from importnb import Notebook
with Notebook():
    from che5 import sim, pid, TF1, TF2, shift

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

from plotly.subplots import make_subplots

from sympy.abc import s
from sympy import exp, Symbol, simplify

In [ ]:
TF1 = exp(-1*s)/(s+1)
TFback1 = exp(-0.2*s)/(s+1)
TFback2 = 2*exp(-0.3*s)/(s+2)
sys1=TF1**5 * pid(0.3,10)
control1=sys1/(1+(sys1*TFback1+sys1*TFback2)/2)
control1 = simplify(control1)

t,y=sim(control1,lambda x: 1., N=5000, dt=0.02)

In [ ]:
simplify(control1)

2*(0.3*s + 0.03)*(s + 1)*(s + 2)*exp(5.5*s)/(2*s*(s + 1)**6*(s + 2)*exp(10.5*s) + (0.3*s + 0.03)*(s + 2)*exp(5.3*s) + (0.6*s + 0.06)*(s + 1)*exp(5.2*s))

In [ ]:
fig=make_subplots()
fig.add_scatter(x=t,y=y, line_width=2, line_color='green')
fig.update_layout(width=600, height=400, template='plotly_dark', showlegend=False)

In [ ]:
#second order with shifted sinusouidal forcing function
t,y=sim(TF2(2,0.1,0.1),shift(lambda t:np.sin(0.1*t),20), N=1000, dt=0.5)
fig=make_subplots()
fig.add_scatter(x=t,y=y, line_width=2, line_color='green')
fig.update_layout(width=600, height=400, template='plotly_dark', showlegend=False)

In [ ]:
#cascade control
TFinner = exp(-0.1*s)/(s+0.5)
TFinner_sys = pid(1.)*TFinner
TFinner_loop = TFinner_sys/(1+TFinner_sys)

TFouter = exp(-s)/(s+5)
TFouter_sys = pid(100.,0.2)*TFinner_loop * TFouter**5
TFouter_loop = TFouter_sys/(1+TFouter_sys)

control2 = simplify(TFouter_loop)

In [ ]:
control2

(100.0*s + 500.0)/(s*(s + 5)**5*((s + 0.5)*exp(0.1*s) + 1.0)*exp(5.0*s) + 100.0*s + 500.0)

In [ ]:
t,y=sim(control2,lambda x: 1., N=1000, dt=0.1)
fig=make_subplots()
fig.add_scatter(x=t,y=y, line_width=2, line_color='green')
fig.update_layout(width=600, height=400, template='plotly_dark', showlegend=False)